In [14]:
import TableParser
import pandas as pd
import tempfile, zipfile
import os
import numpy as np

In [166]:
class TestTableParser(TableParser.TableParserItem):
    def __init__(self):
        pass
    
    def parse(self, series: pd.Series):
        mistake = {
            'Jan':'01','Feb': '02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sept':'09','Oct':'10','Nov':'11','Dec':'12',
            'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5',
            r'[ -/]|[:-@]|[\[-_]|[{-~]': '-',
        }
        for key, val in mistake.items():
            series = series.str.strip().str.replace(key, val, regex=True)
        return (pd.to_datetime(series, errors="ignore"), [])

    def fix_xlsx(self, in_file):
        tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
        os.close(tmpfd)
        filename = '[Content_Types].xml'
        data = ''
        with zipfile.ZipFile(in_file, 'r') as zin:
            with zipfile.ZipFile(tmp, 'w') as zout:
                for item in zin.infolist():
                    if item.filename != filename:
                        zout.writestr(item, zin.read(item.filename))
                    else:
                        data = zin.read(filename).decode()
        os.remove(in_file)
        os.rename(tmp, in_file)
        data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
        with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
            zf.writestr(filename, data)

    ###############################################
    # to find the x,y coordinate of row with date #
    
    def find_coordinate(self, x, y,df): # and defind axis
        test_list =[[0,0]]
        count_X_axis,count_Y_axis,main_axis = 0,0,"Y axis"
        
        for i in range(1,df.shape[0]):
            if len(test_list) == 11: 
                break
            for j in range(df.shape[1]):
                if type(df.iloc[i,j]) == type("string") and len(df.iloc[i,j]) > 5:
                    if type(testTableParser.parse(pd.Series([df.iloc[i,j]]))[0][0]) == type(pd.to_datetime('01.01.2021')):
                        test_list.append([i,j])
                        if test_list[-1][1] > test_list[-2][1]:
                            count_X_axis += 1
                        if test_list[-1][0] > test_list[-2][0]:
                            count_Y_axis += 1  
                        if len(test_list) == 11:
                            break
        if count_X_axis > count_Y_axis:
            main_axis = "X axis"
#         print(main_axis,test_list)
        return test_list[1][0],test_list[1][1],main_axis

    def all_action(self,df):
        data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates
        if main_axis == "X axis":
            table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
            date = df.iloc[data_row_num, data_col_num:]  # <--- Series with date (01.03.2016;...)
            date = self.parse(date)
            table.iloc[0] = date[0]
#             data_row_num,data_col_num = 4,3
            list_index =[]
            for i in range(data_row_num,df.shape[0]):
                buffer = ""
                if pd.isnull(df.iloc[i,0]) and len(list_index)!= 0:
                    buffer = "upWord_"
                for j in range(0,data_col_num):
                    buffer +=str(df.iloc[i,j]).replace('\n',' ')+"__"
                list_index.append(buffer.replace("nan__","")[:-2])
                if list_index[-1][:7] == "upWord_":
                    index = list_index[-2].find("_")
                    list_index[-1] = list_index[-2][:index+2]+list_index[-1][7:]
            list_index = pd.Series(list_index)
            table =table.set_index(list_index)
            
        elif main_axis == "Y axis":
            table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
            date = df.iloc[data_row_num:,data_col_num]
            date = self.parse(date)
            table.iloc[0:,0] = date[0]

#         table.columns = table.iloc[0] # <--- change column names
#         table = table.iloc[1:, :] # <--- remove first row
        return table

In [167]:
testTableParser = TestTableParser()
# testTableParser.fix_xlsx('file1.xlsx')
df = pd.read_excel("file1.xlsx")
testTableParser.all_action(df)

,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,...,Unnamed: 917,Unnamed: 918,Unnamed: 919,Unnamed: 920,Unnamed: 921,Unnamed: 922,Unnamed: 923,Unnamed: 924,Unnamed: 925,Unnamed: 926
Дата на начало периода,2016-01-03 00:00:00,2016-02-03 00:00:00,2016-03-03 00:00:00,2016-04-03 00:00:00,2016-05-03 00:00:00,2016-06-03 00:00:00,2016-07-03 00:00:00,2016-08-03 00:00:00,2016-09-03 00:00:00,2016-10-03 00:00:00,...,2018-05-12 00:00:00,2018-06-12 00:00:00,2018-07-12 00:00:00,2018-08-12 00:00:00,2018-09-12 00:00:00,2018-10-12 00:00:00,2018-11-12 00:00:00,2018-12-12 00:00:00,2018-12-13 00:00:00,2018-12-14 00:00:00
Наличие кур-несушек на начало периода__гол.,24490,24486,24474,24470,24469,24469,24462,24462,24444,24439,...,30036,30032,30026,30009,30009,29999,29995,18092,7119,
Пало__норм__гол.,98,98,98,122,122,122,122,122,122,122,...,150,150,150,150,150,150,150,90,107,
Пало__факт__гол.,10,4,12,4,1,,7,,18,5,...,5,4,6,17,,10,4,5,7,19
Пало__норм__%,0.40,0.40,0.40,0.50,0.50,0.50,0.50,0.50,0.50,0.50,...,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,1.50,1.50
Пало__факт__%,0.04,0.02,0.05,0.02,,,0.03,,0.07,0.02,...,0.02,0.01,0.02,0.06,,0.03,0.01,0.03,0.10,
Расход корма__ц,2900.00,2860.00,2900.00,2860.00,2880.00,2000.00,2900.00,2860.00,2840.00,2860.00,...,3500.00,3600.00,3520.00,,,,,,,
Количество кормодней,24490,24486,24474,24470,24469,24469,24462,24462,24444,24439,...,30036,30032,30026,30009,30009,29999,29995,18092,7119,
Расход корма на голову__норм__г,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,...,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0
Расход корма на голову__факт__г,11841.6,11680.1,11849.3,11687.8,11770.0,8173.6,11855.1,11691.6,11618.4,11702.6,...,11652.7,11987.2,11723.2,,,,,,,


In [4]:
testTableParser.parse(pd.Series(['01.02.2002']))[0][0]

Timestamp('2002-01-02 00:00:00')

In [5]:
# pd.to_datetime('01.02.2002')
df.shape

(30, 927)

0                     Дата на начало периода__nan
1     Наличие кур-несушек на начало периода__гол.
2                                Пало__норм__гол.
3                                Пало__факт__гол.
4                                   Пало__норм__%
5                                   Пало__факт__%
6                                 Расход корма__ц
7                       Количество кормодней__nan
8                 Расход корма на голову__норм__г
9                 Расход корма на голову__факт__г
10          Расход корма на 1000 шт яиц__норм__кг
11          Расход корма на 1000 шт яиц__факт__кг
12       Расход корма на 1 кг яйцемассы__норм__кг
13       Расход корма на 1 кг яйцемассы__факт__кг
14                          Валовый сбор яиц__шт.
15                                  Яйцемасса__кг
16                 Среднемесячное поголовье__гол.
17     Яиц на начальную курицу-несушку__норм__шт.
18     Яиц на начальную курицу-несушку__факт__шт.
19       Яиц на среднюю курицу-несушку__норм__шт.


In [119]:
pd.isnull(df.iloc[3,1])

True

In [48]:
df.iloc[5,0]

'Наличие кур-несушек\nна начало периода'

In [67]:
for j in range(0,data_col_num):
    print(str(df.iloc[4,j]))

Дата на начало периода
nan
nan


In [77]:
# for i in range(data_row_num,df.shape[0]):
#     print(df.iloc[i,0])
df.iloc[5,0].replace('\n','')

'Наличие кур-несушекна начало периода'

0

In [135]:
print("as_asd".find("_"))

2
